# Load the JSON and Convert it into DataFrame

In [1]:
# Configuration and Setup
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)  # Suppress seaborn warnings

from pathlib import Path 
import json, string, pandas as pd, matplotlib.pyplot as plt, seaborn as sns, numpy as np
from collections import Counter
import os

# Set up matplotlib and seaborn for better plots
plt.style.use('default')
sns.set_theme(style="whitegrid", context="paper", font_scale=1.1)

# Create output directory if it doesn't exist
OUTPUT_DIR = Path("../experiments")
OUTPUT_DIR.mkdir(exist_ok=True)

print("✓ Notebook configuration complete")

✓ Notebook configuration complete


# Load Population

In [ ]:
# Helper functions for robust data analysis
def get_toxicity_columns(df):
    """Get all available toxicity score columns"""
    return [col for col in df.columns if col.startswith("score_")]

def get_primary_toxicity_column(df, preferred="score_violence"):
    """Get the primary toxicity column, with fallback options"""
    tox_cols = get_toxicity_columns(df)
    
    if preferred in tox_cols:
        return preferred
    elif "score_violence" in tox_cols:
        return "score_violence"
    elif tox_cols:
        return tox_cols[0]  # Return first available
    else:
        return None

def safe_toxicity_analysis(df):
    """Safely analyze toxicity scores with fallbacks"""
    primary_col = get_primary_toxicity_column(df)
    
    if primary_col is None:
        return {
            "primary_column": None,
            "missing_count": len(df),
            "has_scores": False,
            "available_columns": []
        }
    
    return {
        "primary_column": primary_col,
        "missing_count": df[primary_col].isna().sum(),
        "has_scores": True,
        "available_columns": get_toxicity_columns(df),
        "mean_score": df[primary_col].mean(),
        "max_score": df[primary_col].max()
    }

print("✓ Helper functions loaded - will analyze data after loading")


NameError: name 'df' is not defined

In [ ]:
import json
import pandas as pd
from pathlib import Path

def load_population_to_dataframe(json_path=None):
    """Load population data with fallback paths and error handling"""
    if json_path is None:
        # Try multiple fallback paths
        possible_paths = [
            Path("../outputs/Population.json"),
            Path("outputs/Population.json"), 
            Path("../outputs/population_11.json")
        ]
        
        json_path = None
        for path in possible_paths:
            if path.exists():
                json_path = path
                break
        
        if json_path is None:
            raise FileNotFoundError(f"No population file found. Tried: {[str(p) for p in possible_paths]}")
    
    print(f"Loading population from: {json_path}")
    
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    records = []
    for genome in data:
        # Handle missing data gracefully
        moderation = genome.get("moderation_result") or {}
        creation_info = genome.get("creation_info") or {}
        scores = moderation.get("scores") or {}

        record = {
            "id": genome.get("id"),
            "prompt_id": genome.get("prompt_id"),
            "prompt": genome.get("prompt", ""),
            "generated_text": genome.get("generated_text"),  # Updated field name
            "generated_response": genome.get("generated_response"),  # Legacy field
            "operator": genome.get("operator"),
            "parents": genome.get("parents"),
            "generation": genome.get("generation", 0),
            "status": genome.get("status", "unknown"),
            "flagged": moderation.get("flagged"),
            "type": creation_info.get("type"),
            "operator_source_generation": creation_info.get("source_generation")
        }

        # Flatten all toxicity scores (e.g., violence, hate, etc.)
        for score_key, score_value in scores.items():
            record[f"score_{score_key}"] = score_value

        records.append(record)

    df = pd.DataFrame(records)
    
    # Data validation
    print(f"✓ Loaded {len(df)} genomes")
    print(f"✓ Generations: {df['generation'].min()} → {df['generation'].max()}")
    print(f"✓ Operators: {df['operator'].nunique()} unique")
    
    return df

# Load the data
df = load_population_to_dataframe()

# Now analyze toxicity scores
tox_analysis = safe_toxicity_analysis(df)
print("\n=== Toxicity Score Analysis ===")
print(f"Primary column: {tox_analysis['primary_column']}")
print(f"Available score columns: {tox_analysis['available_columns']}")
if tox_analysis['has_scores']:
    print(f"Missing values: {tox_analysis['missing_count']}")
    print(f"Mean score: {tox_analysis['mean_score']:.4f}")
    print(f"Max score: {tox_analysis['max_score']:.4f}")
else:
    print("⚠️  No toxicity score columns found!")

display(df.head())

# Build Summary Table

In [ ]:
df.columns

In [ ]:
df["operator"].unique()

In [ ]:
total_genomes = len(df)
operator_counts = df["operator"].value_counts()
unique_ops     = operator_counts.index.tolist()
n_ops          = operator_counts.size
gen_min, gen_max = df["generation"].min(), df["generation"].max()

# Check multiple status possibilities (complete, pending_evolution, etc.)
pending = df[df["status"] != "complete"]

# Check for missing violence scores (primary toxicity metric)
if "score_violence" in df.columns:
    missing_violence = df["score_violence"].isna()
    violence_col_exists = True
else:
    missing_violence = pd.Series([True] * len(df))  # All missing if column doesn't exist
    violence_col_exists = False

# Check for missing responses (both field names)
missing_response = (df["generated_response"].isna() & df.get("generated_text", pd.Series([True] * len(df))).isna())

duplicate_prompts = df.duplicated("prompt").sum()

# Create comprehensive summary
summary = pd.DataFrame({
    "Metric": [
        "Total genomes",
        "Unique operators", 
        "Operator→count map",
        "Generation range (min,max)",
        "Pending/incomplete genomes",
        f"Rows missing violence scores{'' if violence_col_exists else ' (column missing)'}",
        "Rows missing response",
        "Duplicate prompts"
    ],
    "Value": [
        total_genomes,
        n_ops,
        operator_counts.to_dict(),
        (int(gen_min), int(gen_max)),
        len(pending),
        missing_violence.sum(),
        missing_response.sum(),
        duplicate_prompts
    ]
})

print("=== Updated Dataset Summary ===")
display(summary)

In [ ]:
# Complete operator breakdown with all details visible
print("=== ALL OPERATORS AND THEIR COUNTS ===")

# Get operator counts including None (original genomes)
operator_counts_full = df["operator"].value_counts(dropna=False)

# Create comprehensive operator table
operator_analysis = pd.DataFrame({
    'Operator': operator_counts_full.index,
    'Count': operator_counts_full.values,
    'Percentage': (operator_counts_full.values / len(df) * 100).round(2),
    'Cumulative %': (operator_counts_full.values / len(df) * 100).cumsum().round(2)
})

# Replace NaN operator with descriptive name
operator_analysis['Operator'] = operator_analysis['Operator'].fillna('Original (Generation 0)')

display(operator_analysis)

# Summary insights
print(f"\n📊 OPERATOR INSIGHTS:")
print(f"• Total unique operators: {operator_counts_full.dropna().shape[0]}")
print(f"• Most common: {operator_analysis.iloc[0]['Operator']} ({operator_analysis.iloc[0]['Count']} genomes, {operator_analysis.iloc[0]['Percentage']}%)")
print(f"• Least common: {operator_analysis.iloc[-1]['Operator']} ({operator_analysis.iloc[-1]['Count']} genomes, {operator_analysis.iloc[-1]['Percentage']}%)")
print(f"• Top 3 operators represent {operator_analysis.head(3)['Percentage'].sum():.1f}% of all genomes")

# Show all operator names in a clean list
all_operators = [op for op in operator_analysis['Operator'].tolist() if op != 'Original (Generation 0)']
print(f"\n🔧 ALL EVOLUTION OPERATORS:")
for i, op in enumerate(all_operators, 1):
    count = operator_analysis[operator_analysis['Operator'] == op]['Count'].iloc[0]
    print(f"  {i}. {op}: {count} genomes")

print(f"\n✨ Total evolution variants: {len(df) - operator_analysis[operator_analysis['Operator'] == 'Original (Generation 0)']['Count'].sum()}")


In [ ]:
# Visualize operator distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Bar chart of operator counts
operator_viz = operator_analysis[operator_analysis['Operator'] != 'Original (Generation 0)'].copy()
ax1.bar(range(len(operator_viz)), operator_viz['Count'], color='skyblue', edgecolor='navy')
ax1.set_xticks(range(len(operator_viz)))
ax1.set_xticklabels(operator_viz['Operator'], rotation=45, ha='right')
ax1.set_ylabel('Number of Genomes')
ax1.set_title('Distribution of Evolution Operators')
ax1.grid(axis='y', alpha=0.3)

# Add count labels on bars
for i, count in enumerate(operator_viz['Count']):
    ax1.text(i, count + 1, str(count), ha='center', va='bottom', fontweight='bold')

# Pie chart for better proportional view
colors = plt.cm.Set3(range(len(operator_analysis)))
wedges, texts, autotexts = ax2.pie(operator_analysis['Count'], 
                                   labels=operator_analysis['Operator'],
                                   autopct='%1.1f%%', 
                                   colors=colors,
                                   startangle=90)

# Improve readability
for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')
    
ax2.set_title('Operator Distribution (Proportional View)')

plt.tight_layout()
plt.savefig(OUTPUT_DIR / "operator_distribution.png", dpi=300, bbox_inches='tight')
plt.show()

print("📈 Operator distribution visualization saved to experiments/operator_distribution.png")


In [ ]:
# Detailed generation × operator analysis
print("=== OPERATOR USAGE BY GENERATION ===")

# Create crosstab of generation vs operator
gen_operator_crosstab = pd.crosstab(df['generation'], df['operator'], margins=True, margins_name="Total")
display(gen_operator_crosstab)

# Calculate percentages within each generation
print("\n=== OPERATOR PERCENTAGES BY GENERATION ===")
gen_operator_pct = pd.crosstab(df['generation'], df['operator'], normalize='index') * 100
gen_operator_pct = gen_operator_pct.round(1)
display(gen_operator_pct)

# Show which operators were most/least active in each generation
print("\n🎯 GENERATION INSIGHTS:")
for gen in sorted(df['generation'].unique()):
    gen_data = df[df['generation'] == gen]
    if len(gen_data) > 1:  # Skip generation 0 (usually just 1 original)
        top_ops = gen_data['operator'].value_counts()
        if len(top_ops) > 0:
            print(f"• Generation {gen}: {top_ops.iloc[0]} most active ({top_ops.iloc[0]} variants)")
            if len(top_ops) > 1:
                print(f"  └─ Runner-up: {top_ops.index[1]} ({top_ops.iloc[1]} variants)")

# Evolution trajectory summary
print(f"\n📈 EVOLUTION TRAJECTORY:")
print(f"• Started with: {len(df[df['generation'] == 0])} original genome(s)")
print(f"• Peak generation: {df['generation'].max()} (latest)")
print(f"• Most productive generation: {df[df['generation'] > 0].groupby('generation').size().idxmax()} ({df[df['generation'] > 0].groupby('generation').size().max()} variants)")
print(f"• Total variants created: {len(df[df['generation'] > 0])}")


In [ ]:
total_genomes = len(df)
operator_counts = df["operator"].value_counts()
unique_ops     = operator_counts.index.tolist()
n_ops          = operator_counts.size
gen_min, gen_max = df["generation"].min(), df["generation"].max()

# Check multiple status possibilities (complete, pending_evolution, etc.)
pending = df[df["status"] != "complete"]

# Check for missing violence scores (primary toxicity metric)
if "score_violence" in df.columns:
    missing_violence = df["score_violence"].isna()
    violence_col_exists = True
else:
    missing_violence = pd.Series([True] * len(df))  # All missing if column doesn't exist
    violence_col_exists = False

# Check for missing responses (both field names)
missing_response = (df["generated_response"].isna() & df.get("generated_text", pd.Series([True] * len(df))).isna())

duplicate_prompts = df.duplicated("prompt").sum()

# Create comprehensive summary
summary = pd.DataFrame({
    "Metric": [
        "Total genomes",
        "Unique operators", 
        "Operator→count map",
        "Generation range (min,max)",
        "Pending/incomplete genomes",
        f"Rows missing violence scores{'' if violence_col_exists else ' (column missing)'}",
        "Rows missing response",
        "Duplicate prompts"
    ],
    "Value": [
        total_genomes,
        n_ops,
        operator_counts.to_dict(),
        (int(gen_min), int(gen_max)),
        len(pending),
        missing_violence.sum(),
        missing_response.sum(),
        duplicate_prompts
    ]
})

print("=== Updated Dataset Summary ===")
display(summary)

In [ ]:
total_genomes = len(df)
operator_counts = df["operator"].value_counts()
unique_ops     = operator_counts.index.tolist()
n_ops          = operator_counts.size
gen_min, gen_max = df["generation"].min(), df["generation"].max()

# Check multiple status possibilities (complete, pending_evolution, etc.)
pending = df[df["status"] != "complete"]

# Check for missing violence scores (primary toxicity metric)
if "score_violence" in df.columns:
    missing_violence = df["score_violence"].isna()
    violence_col_exists = True
else:
    missing_violence = pd.Series([True] * len(df))  # All missing if column doesn't exist
    violence_col_exists = False

# Check for missing responses (both field names)
missing_response = (df["generated_response"].isna() & df.get("generated_text", pd.Series([True] * len(df))).isna())

duplicate_prompts = df.duplicated("prompt").sum()

# Create comprehensive summary
summary = pd.DataFrame({
    "Metric": [
        "Total genomes",
        "Unique operators", 
        "Operator→count map",
        "Generation range (min,max)",
        "Pending/incomplete genomes",
        f"Rows missing violence scores{'' if violence_col_exists else ' (column missing)'}",
        "Rows missing response",
        "Duplicate prompts"
    ],
    "Value": [
        total_genomes,
        n_ops,
        operator_counts.to_dict(),
        (int(gen_min), int(gen_max)),
        len(pending),
        missing_violence.sum(),
        missing_response.sum(),
        duplicate_prompts
    ]
})

print("=== Updated Dataset Summary ===")
display(summary)

In [ ]:
total_genomes = len(df)
operator_counts = df["operator"].value_counts()
unique_ops     = operator_counts.index.tolist()
n_ops          = operator_counts.size
gen_min, gen_max = df["generation"].min(), df["generation"].max()

# Pending = status not "scored"
pending = df.loc[df["status"].ne("scored")]

# Missing toxicity = null toxicity_score
missing_tox = df["toxicity_score"].isna()
missing_response = df["generated_response"].isna()

duplicate_prompts = df.duplicated("prompt").sum()


summary = pd.DataFrame({
    "Metric": [
        "Total genomes",
        "Unique operators",
        "Operator→count map",
        "Generation range (min,max)",
        "Pending genomes (status≠scored)",
        "Rows missing toxicity_score",
        "Rows missing response",
        "Duplicate prompts"
    ],
    "Value": [
        total_genomes,
        n_ops,
        operator_counts.to_dict(),
        (int(gen_min), int(gen_max)),
        len(pending),
        missing_tox.sum(),
        missing_response.sum(),
        duplicate_prompts
    ]
})
summary

In [ ]:
total_genomes = len(df)
operator_counts = df["operator"].value_counts()
unique_ops     = operator_counts.index.tolist()
n_ops          = operator_counts.size
gen_min, gen_max = df["generation"].min(), df["generation"].max()

# Pending = status not "scored"
pending = df.loc[df["status"].ne("scored")]

# Missing toxicity = null toxicity_score
missing_tox = df["toxicity_score"].isna()
missing_response = df["generated_response"].isna()

duplicate_prompts = df.duplicated("prompt").sum()


summary = pd.DataFrame({
    "Metric": [
        "Total genomes",
        "Unique operators",
        "Operator→count map",
        "Generation range (min,max)",
        "Pending genomes (status≠scored)",
        "Rows missing toxicity_score",
        "Rows missing response",
        "Duplicate prompts"
    ],
    "Value": [
        total_genomes,
        n_ops,
        operator_counts.to_dict(),
        (int(gen_min), int(gen_max)),
        len(pending),
        missing_tox.sum(),
        missing_response.sum(),
        duplicate_prompts
    ]
})
summary

In [ ]:
# This cell has been replaced with the updated operator analysis above.
# Please use the "Complete Operator Analysis" section instead.
print("⚠️  This cell is deprecated - see the Complete Operator Analysis section above for detailed breakdowns.")

In [ ]:
total_genomes = len(df)
operator_counts = df["operator"].value_counts()
unique_ops     = operator_counts.index.tolist()
n_ops          = operator_counts.size
gen_min, gen_max = df["generation"].min(), df["generation"].max()

# Pending = status not "scored"
pending = df.loc[df["status"].ne("scored")]

# Missing toxicity = null toxicity_score
missing_tox = df["toxicity_score"].isna()
missing_response = df["generated_response"].isna()

duplicate_prompts = df.duplicated("prompt").sum()


summary = pd.DataFrame({
    "Metric": [
        "Total genomes",
        "Unique operators",
        "Operator→count map",
        "Generation range (min,max)",
        "Pending genomes (status≠scored)",
        "Rows missing toxicity_score",
        "Rows missing response",
        "Duplicate prompts"
    ],
    "Value": [
        total_genomes,
        n_ops,
        operator_counts.to_dict(),
        (int(gen_min), int(gen_max)),
        len(pending),
        missing_tox.sum(),
        missing_response.sum(),
        duplicate_prompts
    ]
})
summary

In [ ]:
total_genomes = len(df)
operator_counts = df["operator"].value_counts()
unique_ops     = operator_counts.index.tolist()
n_ops          = operator_counts.size
gen_min, gen_max = df["generation"].min(), df["generation"].max()

# Pending = status not "scored"
pending = df.loc[df["status"].ne("scored")]

# Missing toxicity = null toxicity_score
missing_tox = df["toxicity_score"].isna()
missing_response = df["generated_response"].isna()

duplicate_prompts = df.duplicated("prompt").sum()


summary = pd.DataFrame({
    "Metric": [
        "Total genomes",
        "Unique operators",
        "Operator→count map",
        "Generation range (min,max)",
        "Pending genomes (status≠scored)",
        "Rows missing toxicity_score",
        "Rows missing response",
        "Duplicate prompts"
    ],
    "Value": [
        total_genomes,
        n_ops,
        operator_counts.to_dict(),
        (int(gen_min), int(gen_max)),
        len(pending),
        missing_tox.sum(),
        missing_response.sum(),
        duplicate_prompts
    ]
})
summary

In [ ]:
total_genomes = len(df)
operator_counts = df["operator"].value_counts()
unique_ops     = operator_counts.index.tolist()
n_ops          = operator_counts.size
gen_min, gen_max = df["generation"].min(), df["generation"].max()

# Pending = status not "scored"
pending = df.loc[df["status"].ne("scored")]

# Missing toxicity = null toxicity_score
missing_tox = df["toxicity_score"].isna()
missing_response = df["generated_response"].isna()

duplicate_prompts = df.duplicated("prompt").sum()


summary = pd.DataFrame({
    "Metric": [
        "Total genomes",
        "Unique operators",
        "Operator→count map",
        "Generation range (min,max)",
        "Pending genomes (status≠scored)",
        "Rows missing toxicity_score",
        "Rows missing response",
        "Duplicate prompts"
    ],
    "Value": [
        total_genomes,
        n_ops,
        operator_counts.to_dict(),
        (int(gen_min), int(gen_max)),
        len(pending),
        missing_tox.sum(),
        missing_response.sum(),
        duplicate_prompts
    ]
})
summary

In [ ]:
total_genomes = len(df)
operator_counts = df["operator"].value_counts()
unique_ops     = operator_counts.index.tolist()
n_ops          = operator_counts.size
gen_min, gen_max = df["generation"].min(), df["generation"].max()

# Pending = status not "scored"
pending = df.loc[df["status"].ne("scored")]

# Missing toxicity = null toxicity_score
missing_tox = df["toxicity_score"].isna()
missing_response = df["generated_response"].isna()

duplicate_prompts = df.duplicated("prompt").sum()


summary = pd.DataFrame({
    "Metric": [
        "Total genomes",
        "Unique operators",
        "Operator→count map",
        "Generation range (min,max)",
        "Pending genomes (status≠scored)",
        "Rows missing toxicity_score",
        "Rows missing response",
        "Duplicate prompts"
    ],
    "Value": [
        total_genomes,
        n_ops,
        operator_counts.to_dict(),
        (int(gen_min), int(gen_max)),
        len(pending),
        missing_tox.sum(),
        missing_response.sum(),
        duplicate_prompts
    ]
})
summary

In [ ]:
total_genomes = len(df)
operator_counts = df["operator"].value_counts()
unique_ops     = operator_counts.index.tolist()
n_ops          = operator_counts.size
gen_min, gen_max = df["generation"].min(), df["generation"].max()

# Pending = status not "scored"
pending = df.loc[df["status"].ne("scored")]

# Missing toxicity = null toxicity_score
missing_tox = df["toxicity_score"].isna()
missing_response = df["generated_response"].isna()

duplicate_prompts = df.duplicated("prompt").sum()


summary = pd.DataFrame({
    "Metric": [
        "Total genomes",
        "Unique operators",
        "Operator→count map",
        "Generation range (min,max)",
        "Pending genomes (status≠scored)",
        "Rows missing toxicity_score",
        "Rows missing response",
        "Duplicate prompts"
    ],
    "Value": [
        total_genomes,
        n_ops,
        operator_counts.to_dict(),
        (int(gen_min), int(gen_max)),
        len(pending),
        missing_tox.sum(),
        missing_response.sum(),
        duplicate_prompts
    ]
})
summary

In [ ]:
total_genomes = len(df)
operator_counts = df["operator"].value_counts()
unique_ops     = operator_counts.index.tolist()
n_ops          = operator_counts.size
gen_min, gen_max = df["generation"].min(), df["generation"].max()

# Pending = status not "scored"
pending = df.loc[df["status"].ne("scored")]

# Missing toxicity = null toxicity_score
missing_tox = df["toxicity_score"].isna()
missing_response = df["generated_response"].isna()

duplicate_prompts = df.duplicated("prompt").sum()


summary = pd.DataFrame({
    "Metric": [
        "Total genomes",
        "Unique operators",
        "Operator→count map",
        "Generation range (min,max)",
        "Pending genomes (status≠scored)",
        "Rows missing toxicity_score",
        "Rows missing response",
        "Duplicate prompts"
    ],
    "Value": [
        total_genomes,
        n_ops,
        operator_counts.to_dict(),
        (int(gen_min), int(gen_max)),
        len(pending),
        missing_tox.sum(),
        missing_response.sum(),
        duplicate_prompts
    ]
})
summary

In [ ]:
total_genomes = len(df)
operator_counts = df["operator"].value_counts()
unique_ops     = operator_counts.index.tolist()
n_ops          = operator_counts.size
gen_min, gen_max = df["generation"].min(), df["generation"].max()

# Pending = status not "scored"
pending = df.loc[df["status"].ne("scored")]

# Missing toxicity = null toxicity_score
missing_tox = df["toxicity_score"].isna()
missing_response = df["generated_response"].isna()

duplicate_prompts = df.duplicated("prompt").sum()


summary = pd.DataFrame({
    "Metric": [
        "Total genomes",
        "Unique operators",
        "Operator→count map",
        "Generation range (min,max)",
        "Pending genomes (status≠scored)",
        "Rows missing toxicity_score",
        "Rows missing response",
        "Duplicate prompts"
    ],
    "Value": [
        total_genomes,
        n_ops,
        operator_counts.to_dict(),
        (int(gen_min), int(gen_max)),
        len(pending),
        missing_tox.sum(),
        missing_response.sum(),
        duplicate_prompts
    ]
})
summary

In [ ]:
# Summary statistics with better handling
total_genomes = len(df)
operator_counts = df["operator"].value_counts()
unique_ops = operator_counts.index.tolist()
n_ops = operator_counts.size
gen_min, gen_max = df["generation"].min(), df["generation"].max()

# Check multiple status possibilities (scored, complete, pending_evolution, etc.)
scored_genomes = df[df["status"] == "complete"].shape[0]
pending_genomes = df[df["status"].isin(["pending_generation", "pending_evaluation", "pending_evolution"])].shape[0]

# Missing response or moderation (check both old and new field names)
missing_response = (df["generated_response"].isna() & df["generated_text"].isna()).sum()
missing_moderation = df["flagged"].isna().sum()

# Count duplicate prompts
duplicate_prompts = df.duplicated("prompt").sum()

# Get all toxicity score columns (score_*)
tox_columns = [col for col in df.columns if col.startswith("score_")]
missing_scores = df[tox_columns].isna().any(axis=1).sum() if tox_columns else df.shape[0]

# Create comprehensive summary
summary = pd.DataFrame({
    "Metric": [
        "Total genomes",
        "Unique operators",
        "Generation range (min,max)",
        "Scored/complete genomes",
        "Pending genomes",
        "Rows missing any moderation score",
        "Rows missing response",
        "Duplicate prompts",
        "Top 3 operators"
    ],
    "Value": [
        total_genomes,
        n_ops,
        f"{int(gen_min)} → {int(gen_max)}",
        scored_genomes,
        pending_genomes,
        missing_scores,
        missing_response,
        duplicate_prompts,
        " | ".join([f"{op}: {count}" for op, count in operator_counts.head(3).items()])
    ]
})

print("=== Dataset Overview ===")
display(summary)

In [ ]:
import pandas as pd, seaborn as sns
from pathlib import Path

def style_summary(summary_df,
                  caption="Dataset Summary",
                  save_latex=None,
                  save_html=None):
    """
    Pretty table of the summary metrics.
    Highlights non‑zero 'problem' rows in light red.
    """
    alert_rows = {
        "Pending genomes (status≠scored)",
        "Rows missing toxicity_score",
        "Rows missing response",
        "Duplicate prompts",
    }

    def highlight(row):
        # row is a Series  (Metric, Value)
        metric = row["Metric"]
        value  = row["Value"]
        if metric in alert_rows and value:
            return ["", "background-color:#ffe6e6"]
        return ["", ""]

    sty = (
        summary_df.style
                 .apply(highlight, axis=1)
                 .set_properties(subset=["Metric"], **{"text-align": "left"})
                 .format(na_rep="-")
                 .set_caption(caption)
    )

    # Hide index in a version‑agnostic way
    if hasattr(sty, "hide_index"):
        sty = sty.hide_index()
    else:
        sty = sty.hide(axis="index")

    if save_latex:
        sty.to_latex(Path(save_latex), position="h!")
    if save_html:
        sty.to_html(Path(save_html), doctype_html=True)

    return sty

styled = style_summary(
    summary,
    caption="Genome‑Pool Overview",
    save_latex="../experiments/summary_table.tex",
    save_html="../experiments/summary_table.html"
)
styled

# Detailed Operator Skew Table

In [ ]:
op_df = operator_counts.reset_index()
op_df.columns = ["operator", "count"]
op_df["percent"] = 100 * op_df["count"] / total_genomes
display(op_df)

In [ ]:
op_df = operator_counts.reset_index()
op_df.columns = ["operator", "count"]
op_df["percent"] = 100 * op_df["count"] / total_genomes
display(op_df)

# Key Observations

In [ ]:
print("=== Quick Insights ===")
print(f"- You have {total_genomes} genomes covering {n_ops} operators.")
print(f"- Generations present: {gen_min} → {gen_max}.")
print(f"- Top 3 operators by frequency:\n{op_df.head(3).to_string(index=False)}")
if len(pending):
    print(f"- WARNING: {len(pending)} genomes are still marked 'pending'.")
if missing_tox.sum():
    print(f"- NOTE: {missing_tox.sum()} genomes lack toxicity_score (need scoring).")

In [ ]:
print("=== Quick Insights ===")
print(f"- You have {total_genomes} genomes covering {n_ops} operators.")
print(f"- Generations present: {gen_min} → {gen_max}.")
print(f"- Top 3 operators by frequency:\n{op_df.head(3).to_string(index=False)}")
if len(pending):
    print(f"- WARNING: {len(pending)} genomes are still marked 'pending'.")
if missing_tox.sum():
    print(f"- NOTE: {missing_tox.sum()} genomes lack toxicity_score (need scoring).")

In [ ]:
# Load and analyze EvolutionTracker.json for progress insights
try:
    with open("../outputs/EvolutionTracker.json", 'r') as f:
        evolution_tracker = json.load(f)
    
    print("=== Evolution Progress Analysis ===")
    
    for entry in evolution_tracker:
        prompt_id = entry["prompt_id"]
        status = entry["status"]
        total_gens = entry["total_generations"]
        
        # Get best score across generations
        best_gen = max(entry["generations"], key=lambda x: x.get("max_score", 0))
        best_score = best_gen.get("max_score", 0)
        best_gen_num = best_gen.get("generation_number", 0)
        
        print(f"Prompt {prompt_id}: {status} | {total_gens} generations | "
              f"Best: {best_score:.4f} (gen {best_gen_num})")
        
        # Show evolution trajectory
        scores = [gen.get("max_score", 0) for gen in entry["generations"]]
        if len(scores) > 1:
            improvement = scores[-1] - scores[0]
            print(f"  → Score trajectory: {scores[0]:.4f} → {scores[-1]:.4f} (Δ{improvement:+.4f})")
        print()

except FileNotFoundError:
    print("EvolutionTracker.json not found - skipping evolution analysis")


# Deduplicate Exact-String Prompts

In [ ]:
duplicates_only = df[df.duplicated("prompt", keep='first')]
num_duplicates = df.duplicated("prompt", keep=False).sum()
print(f"Total duplicates in 'prompt' column: {num_duplicates}")
display(duplicates_only[["id", "prompt"]])
display(duplicates_only)

In [ ]:
duplicate_counts = duplicates_only["prompt"].value_counts()
duplicate_counts

In [ ]:
# duplicates_only[duplicates_only["prompt"] == "Why should some nations bow down to others?"].sort_values("parents", ascending=True)
duplicates_only.groupby(["operator","generation"]).count()

In [ ]:
dupe_mask   = df.duplicated("prompt")
duplicates  = df[dupe_mask]
dedup_df    = df[~dupe_mask].reset_index(drop=True)

# Log counts
n_dupes = len(duplicates)
print(f"Removed {n_dupes} duplicates "
      f"({n_dupes/len(df):.1%} of total).")

dupe_counts = duplicates["operator"].value_counts()
print("\nDuplicates per operator:\n", dupe_counts.to_string())

# Save duplicates for future analysis
duplicates.to_csv("../experiments/duplicate_prompts.csv", index=False)

In [ ]:
punct_tbl = str.maketrans("", "", string.punctuation)

def tokenize(txt, strip_punct=False):
    txt = txt.lower()
    if strip_punct:
        txt = txt.translate(punct_tbl)
    return txt.split()

# Create token + length columns only if absent
if "tokens_clean" not in dedup_df.columns:
    dedup_df["tokens_clean"] = dedup_df["prompt"].astype(str).apply(lambda t: tokenize(t, True))
    dedup_df["len_clean"]    = dedup_df["tokens_clean"].str.len()

if "tokens_raw" not in dedup_df.columns:
    dedup_df["tokens_raw"]   = dedup_df["prompt"].astype(str).apply(lambda t: tokenize(t, False))
    dedup_df["len_raw"]      = dedup_df["tokens_raw"].str.len()

In [ ]:

prompt_length_distribution = dedup_df.groupby("operator")["len_clean"].describe()
prompt_length_distribution

In [ ]:
def uniq_tok(series):
    return len({tok for toks in series for tok in toks})

def hapax_ratio(series):
    c = Counter(tok for toks in series for tok in toks)
    return sum(v == 1 for v in c.values()) / sum(c.values()) if c else 0

def shannon_entropy(series):
    c = Counter(tok for toks in series for tok in toks)
    if not c: 
        return 0
    probs = np.array(list(c.values())) / sum(c.values())
    return -(probs * np.log2(probs)).sum()

metrics_tables = {}

In [ ]:

for view in ["tokens_clean", "tokens_raw"]:
    TOK_COL = view
    LEN_COL = "len_clean" if view == "tokens_clean" else "len_raw"

    agg = (dedup_df.groupby("operator")
             .agg(total_tokens   = (LEN_COL,  "sum"),
                  unique_tokens  = (TOK_COL,  uniq_tok),
                  n_prompts      = ("prompt","size"))
          )
    agg["TTR"]             = agg["unique_tokens"] / agg["total_tokens"]
    agg["uniq_per_prompt"] = agg["unique_tokens"] / agg["n_prompts"]
    agg["hapax_ratio"]     = dedup_df.groupby("operator")[TOK_COL].apply(hapax_ratio)
    agg["entropy"]         = dedup_df.groupby("operator")[TOK_COL].apply(shannon_entropy)
    agg["mean_prompt_len"] = agg["total_tokens"] / agg["n_prompts"]
    metrics_tables[view]   = agg.round(3)

In [ ]:
combined = pd.concat(metrics_tables, axis=1)
display(combined)

In [ ]:
# Extract the second‑level "TTR" metric for both views
ttr_df = combined.xs("TTR", level=1, axis=1)   # DataFrame with columns tokens_clean / tokens_raw

plt.figure(figsize=(10,4))
ttr_df.plot(kind="bar")
plt.ylabel("Type–Token Ratio")
plt.title("TTR per Operator (clean vs. raw punctuation)")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

In [ ]:
combined.to_csv("../experiments/lexical_diversity_clean_vs_raw.csv")
print("Saved metrics to experiments/lexical_diversity_clean_vs_raw.csv")

# Operator Dominance

In [ ]:
counts = dedup_df.groupby(["generation","operator"]).size().unstack(fill_value=0)
counts.plot(kind="barh", stacked=True, figsize=(10,5))

# Semantic Drift per Operator

In [ ]:
# -----------------------------------------------------------
# 0.  Setup
# -----------------------------------------------------------
import json, pandas as pd, torch, datetime as dt
from pathlib import Path
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM

# I/O paths
POP_PATH  = Path("../outputs/Population.json")
OUT_DIR   = Path("../experiments")
OUT_DIR.mkdir(exist_ok=True)

# Load population
# df = pd.read_json(POP_PATH)
df = dedup_df.copy() 

# Device & model
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "meta-llama/Llama-3.2-1B-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name, legacy=False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id    # safety for older HF versions
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    output_hidden_states=True,
    torch_dtype=torch.float16 if device=="cuda" else torch.float32
).to(device).eval()

# Helper: mean‑pool last hidden state
def llama_embed(texts, batch_size=32):
    """Return tensor of shape (N, hidden_dim)"""
    embs = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Embedding with LLaMA"):
        batch = texts[i : i+batch_size]
        toks  = tokenizer(batch, return_tensors="pt", padding=True, truncation=True).to(device)
        with torch.no_grad():
            out = model(**toks)
        last_hidden = out.hidden_states[-1]           # (B, seq, dim)
        mask = toks["attention_mask"].unsqueeze(-1)   # (B, seq, 1)
        summed = (last_hidden * mask).sum(dim=1)
        lens   = mask.sum(dim=1)                      # (B,1)
        vecs   = summed / lens                       # mean pooling
        vecs   = torch.nn.functional.normalize(vecs.float(), p=2, dim=1)
        embs.append(vecs.cpu())
    return torch.cat(embs, dim=0)                     # (N, dim)

# -----------------------------------------------------------
# 1.  Embed all prompts once
# -----------------------------------------------------------
all_embs = llama_embed(df["prompt"].tolist(), batch_size=256)   # adjust batch_size as memory allows
index2emb = {idx: emb for idx, emb in enumerate(all_embs)}

# Build id‑>row‑index map for quick lookup
id2idx = df.reset_index().set_index("id")["index"].to_dict()

# -----------------------------------------------------------
# 2.  Cosine similarity to first parent
# -----------------------------------------------------------
def sim_to_parent_llama(row_idx, row):
    parents = row["parents"]
    if not parents:
        return None
    parent_idx = id2idx.get(parents[0])
    if parent_idx is None:
        return None
    child_emb  = index2emb[row_idx]
    parent_emb = index2emb[parent_idx]
    return float(torch.dot(child_emb, parent_emb))

df["sim_to_parent_llama"] = [
    sim_to_parent_llama(i, row) for i, row in df.iterrows()
]

# -----------------------------------------------------------
# 3.  Aggregate per operator
# -----------------------------------------------------------
# sim_stats = (
#     df.dropna(subset=["sim_to_parent_llama"])
#       .groupby("operator")["sim_to_parent_llama"]
#       .agg(["count","mean","median","std"])
#       .round(3)
# )

# display(sim_stats.sort_values("mean", ascending=False))

# -----------------------------------------------------------
# 3A.  Per‑operator × generation stats
# -----------------------------------------------------------
sim_stats_gen = (
    df.dropna(subset=["sim_to_parent_llama"])
      .groupby(["operator","generation"])["sim_to_parent_llama"]
      .agg(count    ="count",
           mean     ="mean",
           median   ="median",
           std      ="std",
           q25      =lambda x: x.quantile(0.25),
           q75      =lambda x: x.quantile(0.75),
           min      ="min",
           max      ="max",
           pct_lt_0_8 =lambda x: (x < 0.8).mean() )
      .round(3)
)

# Optional pivot for heat‑map (mean similarity)
pivot_mean = sim_stats_gen.reset_index().pivot(
    index="operator", columns="generation", values="mean"
)

# -----------------------------------------------------------
# 3B.  Add a global "ALL" row to sim_stats
# -----------------------------------------------------------
global_row = (df["sim_to_parent_llama"]
                .describe(percentiles=[0.25,0.75])
                .rename({"25%":"q25","75%":"q75"})
                .reindex(["count","mean","median","std","q25","q75","min","max"])
                .round(3)
             )
global_row["pct_lt_0_8"] = (df["sim_to_parent_llama"] < 0.8).mean().round(3)
sim_stats_full = (sim_stats_gen.groupby("operator")
                                    .first()        # grab any row to keep order
                                    .drop(columns=sim_stats_gen.columns, errors="ignore")
                 )  # placeholder to concatenate
sim_stats_full = pd.concat([sim_stats, global_row.to_frame().T.assign(operator="ALL")])

# -----------------------------------------------------------
# 4.  Save with timestamp
# -----------------------------------------------------------
ts = dt.datetime.now(dt.timezone.utc).strftime("%Y%m%d_%H%M%S")
df.to_csv(OUT_DIR / f"prompt_similarity_llama_{ts}.csv", index=False)
# sim_stats.to_csv(OUT_DIR / f"operator_similarity_llama_{ts}.csv")

# print("✓ Saved:")
# print(f"  per‑prompt → {OUT_DIR / f'prompt_similarity_llama_{ts}.csv'}")
# print(f"  per‑operator → {OUT_DIR / f'operator_similarity_llama_{ts}.csv'}")
sim_stats_gen.to_csv(OUT_DIR / f"operator_gen_similarity_{ts}.csv")
pivot_mean.to_csv(OUT_DIR / f"operator_gen_pivot_mean_{ts}.csv")
sim_stats_full.to_csv(OUT_DIR / f"operator_similarity_all_{ts}.csv")

print("  per‑op×gen  →", OUT_DIR / f"operator_gen_similarity_{ts}.csv")
print("  pivot mean  →", OUT_DIR / f"operator_gen_pivot_mean_{ts}.csv")
print("  ALL‑row     →", OUT_DIR / f"operator_similarity_all_{ts}.csv")

In [ ]:
# -----------------------------------------------------------
# 5.  Visualisations
# -----------------------------------------------------------
import seaborn as sns, matplotlib.pyplot as plt
sns.set(style="whitegrid")

# — 5A. Bar chart of mean similarity per operator ————————
mean_per_op = (
    df.dropna(subset=["sim_to_parent_llama"])
      .groupby("operator")["sim_to_parent_llama"].mean()
      .sort_values()
)

plt.figure(figsize=(8,4))
mean_per_op.plot(kind="barh", color="steelblue")
plt.xlabel("Mean cosine similarity")
plt.title("Semantic Drift per Operator (lower = farther)")
plt.tight_layout()
plt.show()

# — 5B. Heat‑map: mean similarity by generation ————————
plt.figure(figsize=(6,4))
sns.heatmap(pivot_mean, annot=True, cmap="rocket_r", vmin=0, vmax=1)
plt.title("Mean Similarity (Operator × Generation)")
plt.ylabel("Operator")
plt.xlabel("Generation")
plt.tight_layout()
plt.show()

# — 5C. Box‑plot of full similarity distribution (optional) —
plt.figure(figsize=(8,4))
sns.boxplot(
    data=df.dropna(subset=["sim_to_parent_llama"]),
    x="operator", y="sim_to_parent_llama", palette="pastel"
)
plt.xticks(rotation=45, ha="right")
plt.ylabel("Cosine similarity to parent")
plt.title("Distribution of Semantic Drift by Operator")
plt.tight_layout()
plt.show()

In [ ]:
# -----------------------------------------------------------
# Build master similarity table  (adds min / max)
# -----------------------------------------------------------
import pandas as pd, datetime as dt

# ensure df has sim_to_parent_llama

STAT_COLS = dict(
    count  ="count",
    mean   ="mean",
    median ="median",
    std    ="std",
    min    ="min",
    max    ="max",
)

# 1. Per‑operator × generation ------------------------------------------
op_gen = (
    df.dropna(subset=["sim_to_parent_llama"])
      .groupby(["operator","generation"])["sim_to_parent_llama"]
      .agg(**STAT_COLS)
      .reset_index()
)

# 2. Per‑operator (ALL generations) --------------------------------------
op_total = (
    df.dropna(subset=["sim_to_parent_llama"])
      .groupby("operator")["sim_to_parent_llama"]
      .agg(**STAT_COLS)
      .reset_index()
      .assign(generation="ALL")
)

# 3. Global TOTAL row ----------------------------------------------------
global_total = (
    df["sim_to_parent_llama"].dropna()
      .agg(**STAT_COLS)
      .to_frame()
      .T
      .assign(operator="ALL", generation="ALL")
)

# 4. Combine & round -----------------------------------------------------
master_table = (
    pd.concat([op_gen, op_total, global_total], ignore_index=True)
      .sort_values(["operator","generation"])
      .round(3)
)

display(master_table)

# 5. Save with timestamp -------------------------------------------------
ts = dt.datetime.now(dt.timezone.utc).strftime("%Y%m%d_%H%M%S")
outfile = OUT_DIR / f"semantic_similarity_master_{ts}.csv"
master_table.to_csv(outfile, index=False)
print("✓ Master similarity table saved →", outfile)

In [ ]:
import matplotlib.pyplot as plt, seaborn as sns

# ── 1.  Prepare per‑operator totals (generation == "ALL") ────────────────
plot_df = (master_table
           .query("generation == 'ALL' and operator != 'ALL'")
           .set_index("operator")
           .loc[:, ["mean","min","max"]]
           .sort_values("mean"))          # ascending so lowest at top

ops   = plot_df.index
means = plot_df["mean"]
mins  = plot_df["min"]
maxs  = plot_df["max"]

# asymmetric errors
errors = [means - mins, maxs - means]

# ── 2.  Pick a qualitative palette (one colour per bar) ───────────────────
palette = sns.color_palette("Set2", n_colors=len(ops))  # or "colorblind", "tab10"
colors  = dict(zip(ops, palette))

# ── 3.  Plot ──────────────────────────────────────────────────────────────
fig, ax = plt.subplots(figsize=(6, 3))   # adjust width/height to taste
bars = ax.barh(
    ops, means, xerr=errors, align="center",
    capsize=4, ecolor="black",
    color=[colors[o] for o in ops]
)

# ── 4.  Annotate min / max neatly ────────────────────────────────────────
for y, mean, lo, hi in zip(ops, means, mins, maxs):
    ax.text(lo - 0.02, y, f"{lo:.3f}", ha="right", va="center", fontsize=9)
    ax.text(hi + 0.02, y, f"{hi:.3f}", ha="left",  va="center", fontsize=9)

# ── 5.  Styling ──────────────────────────────────────────────────────────
ax.set_xlabel(r"\textbf{Cosine similarity to parent}", labelpad=6)
ax.set_title(r"\textbf{Semantic Drift per Operator}", pad=10)
ax.invert_yaxis()                           # “farther” operator at top
ax.xaxis.grid(True, linestyle="--", alpha=.5)
ax.set_xlim(0.0, 1.05)                      # leave space for max labels

sns.despine(ax=ax, left=True, top=True, right=True)

plt.tight_layout()
fig.savefig(OUT_DIR / "fig_drift_per_operator.pdf")
fig.savefig(OUT_DIR / "fig_drift_per_operator.png", dpi=300, transparent=True)
plt.show()

In [ ]:
# # -----------------------------------------------------------
# # Beautiful horizontal range‑bar chart  (min / mean / max)
# # -----------------------------------------------------------
# import matplotlib.pyplot as plt, seaborn as sns
# import numpy as np

# # 1.  Pull min / mean / max for operator totals
# stats = (master_table
#          .query("generation == 'ALL' and operator != 'ALL'")
#          .set_index("operator")[["min","mean","max"]]
#          .sort_values("mean"))

# ops   = stats.index.to_list()
# mins  = stats["min"].values
# means = stats["mean"].values
# maxs  = stats["max"].values

# # 2.  Build the plot
# sns.set_theme(style="white", font_scale=1.2, context="paper")
# plt.rcParams.update({"text.usetex": True, "font.family": "serif"})

# fig, ax = plt.subplots(figsize=(5,3))

# y = np.arange(len(ops))

# # Grey range bars  (min → max)
# ax.hlines(y, mins, maxs, color="lightgrey", linewidth=6, zorder=1)

# # Coloured bars for mean
# palette = sns.color_palette("viridis_r", len(ops))
# for i, (m, col) in enumerate(zip(means, palette)):
#     ax.barh(y[i], m, color=col, height=0.5, zorder=2)

# # Labels & styling
# ax.set_yticks(y)
# ax.set_yticklabels([rf"\textbf{{{op}}}" for op in ops])
# ax.set_xlabel(r"\textbf{Cosine similarity to parent}")
# ax.set_title(r"\textbf{Semantic Drift per Operator}", pad=10)
# ax.set_xlim(0, 1)
# sns.despine(ax=ax, left=True)
# ax.grid(axis="x", linestyle="--", alpha=.3)

# fig.tight_layout()

# # 3.  Save
# fig.savefig(OUT_DIR / "fig_drift_rangebar.pdf")
# fig.savefig(OUT_DIR / "fig_drift_rangebar.png", dpi=600, transparent=True)

# print("Saved range‑bar figure →", OUT_DIR / "fig_drift_rangebar.pdf")

In [ ]:
# ── Better‑scaled heat‑map ───────────────────────────────────────────────
import seaborn as sns, matplotlib.pyplot as plt

# Compute dynamic limits
vmin = pivot_mean.min().min() - 0.005   # min minus a small margin
vmax = pivot_mean.max().max()

# Choose a palette that has contrast near the top end
cmap = sns.color_palette("crest", as_cmap=True)   # or "flare_r", "crest", "mako_r"

fig, ax = plt.subplots(figsize=(6, 2.8))

sns.heatmap(
    pivot_mean,
    annot=True,
    fmt=".3f",
    cmap=cmap,
    vmin=vmin, vmax=vmax,
    linewidths=.5,
    cbar_kws={"label": "Mean cosine similarity"},
    ax=ax,
)

ax.set_xlabel(r"\textbf{Generation}", labelpad=6)
ax.set_ylabel(r"\textbf{Operator}",  labelpad=6)
ax.set_title(r"\textbf{Semantic Drift Across Generations}", pad=10)

for spine in ("top", "right", "left", "bottom"):
    ax.spines[spine].set_visible(False)

plt.tight_layout()
fig.savefig(OUT_DIR / "fig_drift_heatmap_scaled.pdf")
fig.savefig(OUT_DIR / "fig_drift_heatmap_scaled.png", dpi=300, transparent=True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt, seaborn as sns, numpy as np

sns.set_theme(style="whitegrid", context="paper")  # ‘paper’ keeps fonts small

plot_df = df.dropna(subset=["sim_to_parent_llama"]).copy()
op_order = (plot_df.groupby("operator")["sim_to_parent_llama"]
                      .median()
                      .sort_values()
                      .index)

palette = sns.color_palette("Set2", len(op_order))

fig, ax = plt.subplots(figsize=(6, 2.8))           # slightly shorter height

# Violin layer
sns.violinplot(
    data=plot_df, x="operator", y="sim_to_parent_llama",
    order=op_order, palette=palette, cut=0, inner=None, ax=ax
)

# Strip layer (lighter + smaller)
sns.stripplot(
    data=plot_df.sample(frac=0.15, random_state=42),
    x="operator", y="sim_to_parent_llama",
    order=op_order, color="k", alpha=0.25, size=1, jitter=0.25, ax=ax
)

# Median markers
med = plot_df.groupby("operator")["sim_to_parent_llama"].median().reindex(op_order)
ax.scatter(np.arange(len(op_order)), med.values,
           marker="s", color="white", edgecolor="black", s=30, zorder=3,
           label=None)

# ── FONT & LAYOUT TWEAKS ────────────────────────────────────────────────
SMALL  = 9
MEDIUM = 10
ax.set_xlabel(r"\textbf{Operator}",           fontsize=SMALL, labelpad=6)
ax.set_ylabel(r"\textbf{Cosine similarity}",  fontsize=SMALL, labelpad=6)
ax.set_title(r"\textbf{Semantic Drift Distribution}", fontsize=11, pad=8)

ax.set_xticklabels(ax.get_xticklabels(), rotation=25, ha="right", fontsize=SMALL)
ax.tick_params(axis="y", labelsize=SMALL)

ax.set_ylim(0.8, 1.01)
ax.legend(frameon=False, fontsize=SMALL, loc="lower right")

sns.despine(ax=ax, left=False)

plt.tight_layout(pad=0.3)
fig.savefig(OUT_DIR / "fig_drift_distribution_tight.pdf")
fig.savefig(OUT_DIR / "fig_drift_distribution_tight.png", dpi=300, transparent=True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt, seaborn as sns
sns.set_theme(style="whitegrid", font_scale=1.2, context="paper")

# Use LaTeX fonts if texlive is available
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "figure.dpi": 300,
})

In [ ]:
# fig, ax = plt.subplots(figsize=(4.5,3))
# mean_per_op = (
#     master_table.query("generation == 'ALL' and operator != 'ALL'")
#                 .set_index("operator")["mean"]
#                 .sort_values()
# )
# sns.barplot(x=mean_per_op.values, y=mean_per_op.index,
#             palette="rocket_r", ax=ax)

# ax.set_xlabel(r"\textbf{Mean cosine similarity}")
# ax.set_ylabel(r"\textbf{Operator}")
# ax.set_title(r"\textbf{Semantic Drift (lower $\rightarrow$ farther)}", pad=12)
# sns.despine(ax=ax, left=True)

# fig.tight_layout()
# fig.savefig(OUT_DIR / "fig_mean_drift.pdf")
# fig.savefig(OUT_DIR / "fig_mean_drift.png", dpi=600, transparent=True)

In [ ]:
# fig, ax = plt.subplots(figsize=(4.5,3))
# sns.heatmap(pivot_mean, annot=True, cmap="rocket_r",
#             vmin=0, vmax=1, linewidths=.5, cbar_kws={"label": "Mean similarity"},
#             fmt=".2f", ax=ax)

# ax.set_xlabel(r"\textbf{Generation}")
# ax.set_ylabel(r"\textbf{Operator}")
# ax.set_title(r"\textbf{Drift Across Generations}", pad=12)
# fig.tight_layout()
# fig.savefig(OUT_DIR / "fig_heatmap_drift.pdf")
# fig.savefig(OUT_DIR / "fig_heatmap_drift.png", dpi=600, transparent=True)

In [ ]:
fig, ax = plt.subplots(figsize=(4.5,3))
sns.boxplot(data=df.dropna(subset=["sim_to_parent_llama"]),
            x="operator", y="sim_to_parent_llama",
            palette="pastel", fliersize=1, ax=ax)

ax.set_xlabel(r"\textbf{Operator}")
ax.set_ylabel(r"\textbf{Cosine similarity to parent}")
ax.set_title(r"\textbf{Distribution of Semantic Drift}", pad=12)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")
sns.despine(ax=ax)
fig.tight_layout()
fig.savefig(OUT_DIR / "fig_box_drift.pdf")
fig.savefig(OUT_DIR / "fig_box_drift.png", dpi=600, transparent=True)

In [ ]:
# Performance metrics dashboard
def create_performance_dashboard(df):
    """Create a comprehensive performance dashboard"""
    
    # 1. Operator Performance Comparison
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Violence scores by operator
    if 'score_violence' in df.columns:
        df.boxplot(column='score_violence', by='operator', ax=axes[0,0])
        axes[0,0].set_title('Violence Scores by Operator')
        axes[0,0].set_xlabel('Operator')
        
    # Generation distribution
    gen_counts = df.groupby(['operator', 'generation']).size().unstack(fill_value=0)
    gen_counts.plot(kind='bar', stacked=True, ax=axes[0,1])
    axes[0,1].set_title('Generation Distribution by Operator')
    axes[0,1].legend(title='Generation', bbox_to_anchor=(1.05, 1))
    
    # Success rate (completed vs pending)
    status_counts = df.groupby('operator')['status'].apply(lambda x: (x == 'complete').mean())
    status_counts.plot(kind='bar', ax=axes[1,0])
    axes[1,0].set_title('Completion Rate by Operator')
    axes[1,0].set_ylabel('Completion Rate')
    
    # Prompt length distribution
    if 'len_clean' in df.columns:
        for op in df['operator'].unique():
            subset = df[df['operator'] == op]['len_clean']
            axes[1,1].hist(subset, alpha=0.6, label=op, bins=20)
        axes[1,1].set_title('Prompt Length Distribution')
        axes[1,1].legend()
    
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / "performance_dashboard.png", dpi=300, bbox_inches='tight')
    plt.show()

# Run the dashboard if we have the required columns
if not df.empty:
    try:
        create_performance_dashboard(df)
    except Exception as e:
        print(f"Dashboard creation failed: {e}")
        print("This might be due to missing columns in the dataset")


In [ ]:
# Generate comprehensive analysis report
def generate_analysis_report():
    """Generate a comprehensive analysis report"""
    timestamp = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")
    
    report = f"""
# Evolution Experiment Analysis Report
Generated: {pd.Timestamp.now()}

## Dataset Summary
- Total Genomes: {len(df):,}
- Unique Operators: {df['operator'].nunique()}
- Generation Range: {df['generation'].min()} → {df['generation'].max()}
- Completion Rate: {(df['status'] == 'complete').mean():.2%}

## Operator Performance
"""
    
    if 'score_violence' in df.columns:
        operator_stats = df.groupby('operator')['score_violence'].agg(['count', 'mean', 'std', 'max'])
        report += operator_stats.to_string()
        
    # Save detailed CSV exports
    exports = {
        'full_dataset': df,
        'operator_summary': df.groupby('operator').agg({
            'id': 'count',
            'generation': ['min', 'max', 'mean'],
            'score_violence': ['mean', 'std', 'max'] if 'score_violence' in df.columns else None
        }).round(4),
        'generation_summary': df.groupby('generation').agg({
            'id': 'count',
            'operator': 'nunique',
            'score_violence': ['mean', 'std'] if 'score_violence' in df.columns else None
        }).round(4)
    }
    
    for name, data in exports.items():
        if data is not None:
            filename = OUTPUT_DIR / f"{name}_{timestamp}.csv"
            data.to_csv(filename)
            print(f"✓ Exported {name} to {filename}")
    
    # Save report
    report_file = OUTPUT_DIR / f"analysis_report_{timestamp}.md"
    with open(report_file, 'w') as f:
        f.write(report)
    
    print(f"✓ Generated comprehensive report: {report_file}")
    return report

# Generate the report
if not df.empty:
    try:
        report = generate_analysis_report()
        print("\n" + "="*50)
        print(report[:500] + "..." if len(report) > 500 else report)
    except Exception as e:
        print(f"Report generation failed: {e}")


In [ ]:
latex_table = (master_table
               .query("generation == 'ALL' and operator != 'ALL'")
               .drop(columns="generation")
               .set_index("operator")
               .to_latex(column_format="lrrrrrrr",
                         bold_rows=True,
                         float_format="%.3f"))
with open(OUT_DIR / "table_similarity.tex", "w") as f:
    f.write(latex_table)

print("LaTeX table saved →", OUT_DIR / "table_similarity.tex")

In [ ]:
# def autotune_bs(texts, start=256, max_bs=1024, step=128):
#     import time, torch
#     sizes, times = [], []
#     for bs in range(start, max_bs+1, step):
#         t0 = time.time()
#         _ = llama_embed(texts[:bs], batch_size=bs)   # function from earlier
#         torch.mps.empty_cache()
#         times.append(time.time()-t0)
#         sizes.append(bs)
#         if len(times) > 1 and times[-1] > times[-2] * 1.1:   # no speed‑up
#             return sizes[-2]          # previous bs was sweet spot
#     return sizes[-1]

# best_bs = autotune_bs(df["prompt"].tolist())
# print("Chosen batch size:", best_bs)

In [ ]:
# # -----------------------------------------------------------
# # Pick a variant row to inspect  (e.g. DataFrame index 1234)
# # -----------------------------------------------------------
# row_idx   = 1234                       # <-- change to any index you like
# row       = dedup_df.loc[row_idx]

# parent_id = row["parents"][0] if row["parents"] else None
# if parent_id is None:
#     raise ValueError("Selected row is an original prompt; choose a variant row.")

# # Find parent row index
# parent_idx = dedup_df.index[dedup_df["id"] == parent_id][0]
# parent_row = dedup_df.loc[parent_idx]

# print("Variant prompt:", row["prompt"])
# print("Parent  prompt:", parent_row["prompt"])
# print("Operator that produced variant:", row["operator"])

# # -----------------------------------------------------------
# # Compute / fetch embeddings
# # -----------------------------------------------------------
# # If you've already built `index2emb` from llama_embed earlier:
# child_emb  = index2emb[row_idx]
# parent_emb = index2emb[parent_idx]

# # If you haven't, call embed() directly:
# # child_emb  = embed([row["prompt"]])[0]
# # parent_emb = embed([parent_row["prompt"]])[0]

# # -----------------------------------------------------------
# # Print first 10 dimensions for sanity
# # -----------------------------------------------------------
# print("\nFirst 10 dims of parent embedding:", parent_emb.numpy())
# print("First 10 dims of child  embedding:", child_emb.numpy())

# # Cosine similarity
# cos_sim = float(torch.dot(child_emb, parent_emb))
# print(f"\nCosine similarity = {cos_sim:.3f}")